## Evaluating Uncertainty and Predictive Performance of Probabilistic Models

<b>Raymond Leung, Alexander Lowe, Arman Melkumyan</b><br>
Rio Tinto Centre, Faculty of Engineering<br>
The University of Sydney, 2024

`SPDX-FileCopyrightText: 2024 Raymond Leung and Alexander Lowe <raymond.leung@sydney.edu.au>`<br>
`SPDX-License-Identifier: `[`BSD-3-Clause`](https://opensource.org/license/BSD-3-Clause)<br>


### Demonstration code
This notebook clarifies how the methods described in [README.md](../README.md) are used to build and evaluate probabilistic models. It provides a template for running the experiments for a given domain and inference period. It
- Generates all models of interest: SK, OK, SK-SGS, OK-SGS, GP(L), GP(G) GP-SGS, GP-CRF.
- Computes the histogram, variogram and uncertainty-based statistics (described below) and performs rank analysis.
- Produces graphs and figures, including the kappa-accuracy and interval tightness plots, $\hat{\mu}$, $\hat{\sigma}$, $s(\hat{\mu},\hat{\sigma}\mid\mu_0)$ and variograms.

## <font color="#cc0066">Part 1: Models Construction</font>

In [ ]:
# Required Python modules
import ast
import copy
import numpy as np
import pandas as pd
import skgstat as skg
import os
import pickle
import sys
import time
import warnings

sys.path.append(os.getcwd().replace('notebook', 'code'))

from IPython.display import Image
from collections import OrderedDict
from matplotlib import pyplot as plt
from pdb import set_trace as bp
from sklearn.exceptions import ConvergenceWarning
from scipy.stats import norm

from gstatsim3d_utils import make_scatter_2d, timeit
from gstatsim3d_gaussian_process import GPManager
from gstatsim3d_kriging import KrigingRegression, KrigingManager

from rtc_trend_alignment import compute_any_rotation_and_scaling_matrix
from rtc_evaluation_metrics import *
from rtc_utils import *

### Experiment conditions

In [ ]:
# Mandatory parameters specified by user
#===============================================================================
#  The `inference_prefix` is used to define the modelling period "mA_mB_mC"
#  which indicates an intent to estimate the target variable (copper grade)
#  for a 3 month period starting from month "mA" using previously gathered data
inference_prefix = mA = 4

#  The `domain_id` limits the scope of the regionalised variable to certain
#  spatial domain based on geological conditions established by mine geologists.
domain_id = 2310

#  Optional config overrides
overrides = {
    'inference_type': 'future-bench-prediction',
    'simulation:num': 32, #number of simulations used in our experiments was 128
    'kriging:transform_data': True, #rotate and scale according to domain orientation
    'gp:learning_inference_in_rotated_space': True
}
#===============================================================================

specs = {'mA': mA, 'domain_id': domain_id}
specs.update(overrides)

### Load data

In [ ]:
mA_mB_mC = '%02d_%02d_%02d' % (mA, mA+1, mA+2)
model_names = []
model_exec_times = []
learn_times = []
inference_times = []

# Global configuration
cfg_rs = dict()
default_code_dir = os.getcwd().replace('notebook', 'code')
default_data_dir = default_code_dir.replace('code', 'data')
default_result_dir = default_code_dir.replace('code', 'results')
cfg_rs['info:data_dir'] = specs.get('info:data_dir', default_data_dir)
cfg_rs['info:result_dir'] = specs.get('info:result_dir', default_result_dir)
cfg_rs['gp:associate_plunge_rotation_with_x'] = False
specs.update(cfg_rs)
data_path = f"{cfg_rs['info:data_dir']}/{mA_mB_mC}"

# Check if variogram fitting and GP learning occur in rotated space
subdir = check_learning_rotation_status(specs)
result_path = f"{cfg_rs['info:result_dir']}/{subdir}/{mA_mB_mC}"
figure_path = f"{cfg_rs['info:result_dir']}/{subdir}/{mA_mB_mC}/figs"
os.makedirs(figure_path, exist_ok=True)

# Load data
df_bh = pd.read_csv(f"{data_path}/blastholes_tagged.csv")
df_bh = df_bh.rename(columns = {'EAST': 'X', 'NORTH': 'Y', 'RL': 'Z', 'PL_CU': 'V'})
df_domain_x = pd.DataFrame(columns=['X','Y','Z','V']) #not using exploration assays
df_domain_bh = df_bh[(df_bh['lookup_domain'] == domain_id) & np.isfinite(df_bh['V'].values)]
min_v, max_v = np.min(df_domain_bh['V'].values), np.max(df_domain_bh['V'].values)
R, S = compute_any_rotation_and_scaling_matrix(cfg_rs, domain_id)

# Inference locations
df_k = pd.read_csv(f"{data_path}/blocks_to_estimate_tagged.csv")
df_domain_infer = pd.DataFrame(df_k[df_k['domain'] == domain_id])
ground_truth = df_domain_infer['cu_bh_nn'].values

# Visualise blastholes grades
# - Add `savefile=os.path.join(figure_path, f"blastholes_grade_{domain_id}.pdf")`
#   to save figure as a PDF file. This, however, will suppress the display.
_ = make_scatter_2d(df_domain_bh['X'], df_domain_bh['Y'], df_domain_bh['V'],
                min_v, max_v, symbsiz=50, palette='YlOrRd', cbtitle="Cu grade", symbol='s',
                graphtitle=f"Blastholes Cu grade - known samples for {mA_mB_mC}")

### Configure Kriging and simulation parameters

In [ ]:
# Configure Kriging and simulation parameters
cfg_krige = create_kriging_config(R, S, specs)
print(f"Processing {mA_mB_mC}, domain {domain_id}")
print(f"- Training uses {len(df_domain_bh)} blastholes")
print(f"- Inferencing: {len(df_domain_infer)} blocks")
if subdir == 'learning_rotated':
    print(f"Rotation matrix R={R}\nScaling vector S={S}")
if cfg_krige['simulation:num'] > 256:
    specs['simulation:num'] = cfg_krige['simulation:num'] = 256
    print('The maximum number of simulations is capped at 256')

### Technique 1: OK Sequential Gaussian Simulation

In [ ]:
cfg_oksgs = copy.deepcopy(cfg_krige)
oksgs_abbrev = ''.join([x[0] for x in cfg_oksgs['kriging:type'].split('_')]) + 'sgs'
oksgs_abbrev += 'r' if cfg_oksgs['kriging:transform_data'] else ''
oksgs_csv = f"{result_path}/predictions-{oksgs_abbrev}-{domain_id}.csv"

model_names.append('OK-SGS')
if os.path.isfile(oksgs_csv):
    print(f"Reading {oksgs_csv}")
    cfg_oksgs['bypass_simulation'] = True   # only retrieve meta-data such as column labels
    KrigingManager.kriging_sequential_simulations(df_domain_bh, df_domain_infer, cfg_oksgs)
    df_oksgs = pd.read_csv(oksgs_csv, index_col=0, header=0)
    model_exec_times.append(0)
    learn_times.append(0)
    inference_times.append(0)
else:
    print(f"Running simulation...\nResults will be saved to {oksgs_csv}")
    t0 = time.time()
    with warnings.catch_warnings():
        # Suppress the "DataFrame is highly fragmented" warning.
        # - This is usually the result of calling `frame.insert` many times,
        #   which has poor performance. Consider joining all columns at once
        #   using pd.concat(axis=1) instead.
        warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
        df_oksgs = KrigingManager.kriging_sequential_simulations(df_domain_bh, df_domain_infer, cfg_oksgs)
    model_exec_times.append(time.time() - t0)
    learn_times.append(cfg_oksgs['timing:learn'])
    inference_times.append(cfg_oksgs['timing:inference'])
    print(f"Writing {oksgs_csv}")
    df_oksgs.to_csv(oksgs_csv)

# OK-SGS: Compute mean, stdev using first m simulations
max_simul = cfg_krige['simulation:num']
two_powers = [2**i for i in np.arange(1, int(np.floor(np.log2(max_simul))+1))]
oksgs_mean = {}
oksgs_stdev = {}
oksgs_sim_cols = [x for x in df_oksgs.columns]
oksgs_vals = df_oksgs[oksgs_sim_cols].values
for i in two_powers:
    oksgs_mean[f'from_{i}'] = np.mean(oksgs_vals[:,:i], axis=1)
    oksgs_stdev[f'from_{i}'] = np.std(oksgs_vals[:,:i], axis=1)

# Check dictionary contents following simulations, including random states
print('{}\n'.format(cfg_oksgs.keys()))
print(f"simulation:period_domain_id: {cfg_oksgs['simulation:period_domain_id']}")
print(f"simulation:period_domain_initial_state: {cfg_oksgs['simulation:period_domain_initial_state']}")
print(f"simulation:path_seeds: {cfg_oksgs['simulation:path_seeds']}")

# Check variogram parameters
variogram_props = ['nugget','range','sill','nu','variogram-model','R','S']
print("kriging:transform_data: {}".format(cfg_oksgs['kriging:transform_data']))
print("variogram:params: {}".format(dict(zip(variogram_props, cfg_oksgs['variogram:params']))))

### Technique 2: Ordinary Kriging (without sequential simulation)
- Case 'nst': with normal score transformation
- Case 'raw': without data transformation

In [ ]:
cfg_ok = copy.deepcopy(cfg_krige)
cfg_ok['kriging:type'] = 'ordinary_kriging'
ok_mean, ok_var = {}, {}
ok_col_mean, ok_col_stdev = {}, {}
df_ok = pd.DataFrame()
ok_abbrev = 'ok' + ('r' if cfg_ok['kriging:transform_data'] else '')
ok_csv = f"{result_path}/predictions-{ok_abbrev}-{domain_id}.csv"

if os.path.isfile(ok_csv):
    df_ok = pd.read_csv(ok_csv, index_col=0, header=0)
    (ok_mean, ok_var, ok_col_mean, ok_col_stdev) = \
        extract_raw_nst_predictions(df_ok, return_variance=True)
    model_names += ['OK_raw', 'OK_nst']
    model_exec_times += [0,0]
    learn_times += [0,0]
    inference_times += [0,0]
else:
    for xform, bval in [('raw', False), ('nst', True)]:
        cfg_ok['kriging:apply_normal_score_transform'] = bval
        t0 = time.time()
        ok_mean[xform], ok_var[xform] = \
            KrigingManager.kriging_regression(df_domain_bh, df_domain_infer, cfg_ok)
        model_names.append(f"OK_{xform}")
        model_exec_times.append(time.time() - t0)
        learn_times.append(cfg_ok['timing:learn'])
        inference_times.append(cfg_ok['timing:inference'])
        ok_col_mean[xform] = 'ok_mean' + ('_nst' if xform == 'nst' else '')
        ok_col_stdev[xform] = 'ok_stdev' + ('_nst' if xform == 'nst' else '')
        df_ok[ok_col_mean[xform]] = ok_mean[xform]
        df_ok[ok_col_stdev[xform]] = np.sqrt(np.maximum(ok_var[xform], 0))

    df_ok = df_ok.set_index(df_domain_infer.index) #pd>Index generally not contiguous
    df_ok.to_csv(ok_csv)
    print(f"Writing {ok_csv}")

for xform in ['raw', 'nst']:
    print('OK MSE[{}]: {}'.format(xform, np.mean((ok_mean[xform] - ground_truth)**2)))

### Technique 3: SK Sequential Gaussian Simulation

In [ ]:
cfg_krige['kriging:type'] = 'simple_kriging'
cfg_sksgs = copy.deepcopy(cfg_krige)
cfg_sksgs['bypass_simulation'] = False
sksgs_abbrev = ''.join([x[0] for x in cfg_sksgs['kriging:type'].split('_')]) + 'sgs'
sksgs_abbrev += 'r' if cfg_sksgs['kriging:transform_data'] else ''
sksgs_csv = f"{result_path}/predictions-{sksgs_abbrev}-{domain_id}.csv"

model_names.append('SK-SGS')
if os.path.isfile(sksgs_csv):
    print(f"Reading {sksgs_csv}")
    cfg_sksgs['bypass_simulation'] = True
    KrigingManager.kriging_sequential_simulations(df_domain_bh, df_domain_infer, cfg_sksgs)
    df_sksgs = pd.read_csv(sksgs_csv, index_col=0, header=0)
    model_exec_times.append(0)
    learn_times.append(0)
    inference_times.append(0)
else:
    print(f"Running simulation...\nResults will be saved to {sksgs_csv}")
    t0 = time.time()
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
        df_sksgs = KrigingManager.kriging_sequential_simulations(df_domain_bh, df_domain_infer, cfg_sksgs)
    model_exec_times.append(time.time() - t0)
    learn_times.append(cfg_sksgs['timing:learn'])
    inference_times.append(cfg_sksgs['timing:inference'])
    print(f"Writing {sksgs_csv}")
    df_sksgs.to_csv(sksgs_csv)

sksgs_mean = {}
sksgs_stdev = {}
sksgs_sim_cols = [x for x in df_sksgs.columns]
sksgs_vals = df_sksgs[sksgs_sim_cols].values
for i in two_powers:
    sksgs_mean[f'from_{i}'] = np.mean(sksgs_vals[:,:i], axis=1)
    sksgs_stdev[f'from_{i}'] = np.std(sksgs_vals[:,:i], axis=1)

print("kriging:transform_data: {}".format(cfg_sksgs['kriging:transform_data']))
print("variogram:params: {}".format(dict(zip(variogram_props, cfg_sksgs['variogram:params']))))

### Technique 4: Simple Kriging (without sequential simulation)

In [ ]:
cfg_sk = copy.deepcopy(cfg_krige)
cfg_sk['kriging:type'] = 'simple_kriging'
sk_mean, sk_var = {}, {}
sk_col_mean, sk_col_stdev = {}, {}
df_sk = pd.DataFrame()
sk_abbrev = 'sk' + ('r' if cfg_sk['kriging:transform_data'] else '')
sk_csv = f"{result_path}/predictions-{sk_abbrev}-{domain_id}.csv"

if os.path.isfile(sk_csv):
    df_sk = pd.read_csv(sk_csv, index_col=0, header=0)
    (sk_mean, sk_var, sk_col_mean, sk_col_stdev) = \
        extract_raw_nst_predictions(df_sk, return_variance=True)
    model_names += ['SK_raw', 'SK_nst']
    model_exec_times += [0,0]
    learn_times += [0,0]
    inference_times += [0,0]
else:
    for xform, bval in [('raw', False), ('nst', True)]:
        cfg_sk['kriging:apply_normal_score_transform'] = bval
        t0 = time.time()
        sk_mean[xform], sk_var[xform] = \
            KrigingManager.kriging_regression(df_domain_bh, df_domain_infer, cfg_sk)
        model_names.append(f"SK_{xform}")
        model_exec_times.append(time.time() - t0)
        learn_times.append(cfg_sk['timing:learn'])
        inference_times.append(cfg_sk['timing:inference'])
        sk_col_mean[xform] = 'sk_mean' + ('_nst' if xform == 'nst' else '')
        sk_col_stdev[xform] = 'sk_stdev' + ('_nst' if xform == 'nst' else '')
        df_sk[sk_col_mean[xform]] = sk_mean[xform]
        df_sk[sk_col_stdev[xform]] = np.sqrt(np.maximum(sk_var[xform], 0))

    df_sk = df_sk.set_index(df_domain_infer.index)
    df_sk.to_csv(sk_csv)
    print(f"Writing {sk_csv}")

for xform in ['raw', 'nst']:
    print('SK MSE[{}]: {}'.format(xform, np.mean((sk_mean[xform] - ground_truth)**2)))

### Configure Gaussian Process and simulation parameters
- Refer to doc string in `GPManager.gaussian_process_simulations`

In [ ]:
cfg_gp = create_gaussian_process_config(specs)

### Technique 5: Gaussian Process Regression - Local Mean Approach
- GPR Case 1: with normal score transformation
- GPR Case 2: without data transformation

In [ ]:
method = 'GPR(L)'
gpl_mean, gpl_stdev = {}, {}
gpl_col_mean, gpl_col_stdev = {}, {}
df_gpl = pd.DataFrame()
gpl_abbrev = 'gpl' + ('r' if cfg_gp['gp:learning_inference_in_rotated_space'] else '')
gpl_csv = f"{result_path}/predictions-{gpl_abbrev}-{domain_id}.csv"

if os.path.isfile(gpl_csv):
    print(f"Reading {gpl_csv}")
    df_gpl = pd.read_csv(gpl_csv, index_col=0, header=0)
    (gpl_mean, gpl_stdev, gpl_col_mean, gpl_col_stdev) = \
        extract_raw_nst_predictions(df_gpl, return_variance=False)
    model_names += ['GP(L)_raw', 'GP(L)_nst']
    model_exec_times += [0,0]
    learn_times += [0,0]
    inference_times += [0,0]
else:
    for xform, bval in [('raw', False), ('nst', True)]:
        cfg_gp['gp:apply_normal_score_transform'] = bval
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=ConvergenceWarning)
            t0 = time.time()
            # perform GPR
            gpl_mean[xform], gpl_stdev[xform] = \
                GPManager.gaussian_process_simulations(
                method, df_domain_bh, df_domain_x, df_domain_infer, cfg_gp)
            model_names.append(f"GP(L)_{xform}")
            model_exec_times.append(time.time() - t0)
            learn_times.append(cfg_gp['timing:learn'])
            inference_times.append(cfg_gp['timing:inference'])
            gpl_col_mean[xform] = cfg_gp['gp:mean_col_name']
            gpl_col_stdev[xform] = cfg_gp['gp:stdev_col_name']
            df_gpl[gpl_col_mean[xform]] = gpl_mean[xform]
            df_gpl[gpl_col_stdev[xform]] = gpl_stdev[xform]

    df_gpl = df_gpl.set_index(df_domain_infer.index)
    df_gpl.to_csv(gpl_csv)
    print(f"Writing {gpl_csv}")

for xform in ['raw', 'nst']:
    print('GPR(L) MSE[{}]: {}'.format(xform, np.mean((gpl_mean[xform] - ground_truth)**2)))

### Technique 6: GP Sequential Simulation

In [ ]:
method = 'GP-SGS'
cfg_gp['simulation:bypass'] = False
gpsgs_abbrev = 'gpsgs' + ('r' if cfg_gp['gp:learning_inference_in_rotated_space'] else '')
gpsgs_csv = f"{result_path}/predictions-{gpsgs_abbrev}-{domain_id}.csv"

model_names.append('GP-SGS')
if os.path.isfile(gpsgs_csv):
    print(f"Reading {gpsgs_csv}")
    df_gplsgs = pd.read_csv(gpsgs_csv, index_col=0, header=0)
    cfg_gp['simulation:column_names'] = list(df_gplsgs.columns)
    cfg_gp['simulation:bypass'] = True
    GPManager.gaussian_process_simulations(
        method, df_domain_bh, df_domain_x, df_domain_infer, cfg_gp)
    model_exec_times.append(0)
    learn_times.append(0)
    inference_times.append(0)
else:
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
        print(f"Running simulation...\nResults will be saved to {gpsgs_csv}")
        t0 = time.time()
        df_gplsgs = GPManager.gaussian_process_simulations(
                   method, df_domain_bh, df_domain_x, df_domain_infer, cfg_gp)
        model_exec_times.append(time.time() - t0)
        learn_times.append(cfg_gp['timing:learn'])
        inference_times.append(cfg_gp['timing:inference'])
        print(f"Writing {gpsgs_csv}")
        df_gplsgs.to_csv(gpsgs_csv)

gpsgs_mean = {}
gpsgs_stdev = {}
gpsgs_vals = df_gplsgs[cfg_gp['simulation:column_names']].values

max_simul = cfg_gp['simulation:num']
two_powers = [2**i for i in np.arange(1, int(np.floor(np.log2(max_simul))+1))]

for i in two_powers:
    gpsgs_mean[f'from_{i}'] = np.mean(gpsgs_vals[:,:i], axis=1)
    gpsgs_stdev[f'from_{i}'] = np.std(gpsgs_vals[:,:i], axis=1)

min_vsd = min([min(s) for s in gpsgs_stdev.values()])
max_vsd = max([max(s) for s in gpsgs_stdev.values()])

# Check dictionary contents following simulations, including random states
print('{}\n'.format(cfg_gp.keys()))
print(f"simulation:period_domain_id: {cfg_gp['simulation:period_domain_id']}")
print(f"simulation:period_domain_initial_state: {cfg_gp['simulation:period_domain_initial_state']}")
print(f"simulation:path_seeds: {cfg_gp['simulation:path_seeds']}")

### Technique 7: Gaussian Process Regression - Global Mean Approach

In [ ]:
method = 'GPR(G)'
gpg_mean, gpg_stdev = {}, {}
gpg_col_mean, gpg_col_stdev = {}, {}
df_gpg = pd.DataFrame()
gpg_abbrev = 'gpg' + ('r' if cfg_gp['gp:learning_inference_in_rotated_space'] else '')
gpg_csv = f"{result_path}/predictions-{gpg_abbrev}-{domain_id}.csv"

if os.path.isfile(gpg_csv):
    print(f"Reading {gpg_csv}")
    df_gpg = pd.read_csv(gpg_csv, index_col=0, header=0)
    (gpg_mean, gpg_stdev, gpg_col_mean, gpg_col_stdev) = \
        extract_raw_nst_predictions(df_gpg, return_variance=False)
    model_names += ['GP(G)_raw', 'GP(G)_nst']
    model_exec_times += [0,0]
    learn_times += [0,0]
    inference_times += [0,0]
else:
    for xform, bval in [('raw', False), ('nst', True)]:
        cfg_gp['gp:apply_normal_score_transform'] = bval
        with warnings.catch_warnings():
            warnings.filterwarnings("ignore", category=ConvergenceWarning)
            t0 = time.time()
            # perform GPR
            gpg_mean[xform], gpg_stdev[xform] = \
                GPManager.gaussian_process_simulations(
                method, df_domain_bh, df_domain_x, df_domain_infer, cfg_gp)
            model_names.append(f"GP(G)_{xform}")
            model_exec_times.append(time.time() - t0)
            learn_times.append(cfg_gp['timing:learn'])
            inference_times.append(cfg_gp['timing:inference'])
            gpg_col_mean[xform] = cfg_gp['gp:mean_col_name']
            gpg_col_stdev[xform] = cfg_gp['gp:stdev_col_name']
            df_gpg[gpg_col_mean[xform]] = gpg_mean[xform]
            df_gpg[gpg_col_stdev[xform]] = gpg_stdev[xform]

    df_gpg = df_gpg.set_index(df_domain_infer.index)
    df_gpg.to_csv(gpg_csv)
    print(f"Writing {gpg_csv}")

for xform in ['raw', 'nst']:
    print('GPR(G) MSE[{}]: {}'.format(xform, np.mean((gpg_mean[xform] - ground_truth)**2)))

### Technique 8: GP Spatially Correlated (or Cholesky) Random Field

In [ ]:
method = 'GP-CRF'
gpcrf_abbrev = 'gpcrf' + ('r' if cfg_gp['gp:learning_inference_in_rotated_space'] else '')
gpcrf_csv = f"{result_path}/predictions-{gpcrf_abbrev}-{domain_id}.csv"

model_names.append('GP-CRF')
if os.path.isfile(gpcrf_csv):
    print(f"Reading {gpcrf_csv}")
    df_gplcrf = pd.read_csv(gpcrf_csv, index_col=0, header=0)
    cfg_gp['simulation:column_names'] = list(df_gplcrf.columns)
    cfg_gp['simulation:bypass'] = True
    GPManager.gaussian_process_simulations(method, df_domain_bh, df_domain_x, df_domain_infer, cfg_gp)
    model_exec_times.append(0)
    learn_times.append(0)
    inference_times.append(0)
else:
    with warnings.catch_warnings():
        warnings.filterwarnings("ignore", category=pd.errors.PerformanceWarning)
        print(f"Running simulation...\nResults will be saved to {gpcrf_csv}")
        t0 = time.time()
        df_gplcrf = GPManager.gaussian_process_simulations(
                   method, df_domain_bh, df_domain_x, df_domain_infer, cfg_gp)
        model_exec_times.append(time.time() - t0)
        learn_times.append(cfg_gp['timing:learn'])
        inference_times.append(cfg_gp['timing:inference'])
        print(f"Writing {gpcrf_csv}")
        df_gplcrf.to_csv(gpcrf_csv)

gpcrf_mean = {}
gpcrf_stdev = {}
gpcrf_vals = df_gplcrf[cfg_gp['simulation:column_names']].values
for i in two_powers:
    gpcrf_mean[f'from_{i}'] = np.mean(gpcrf_vals[:,:i], axis=1)
    gpcrf_stdev[f'from_{i}'] = np.std(gpcrf_vals[:,:i], axis=1)

### Record execution times

In [ ]:
num = len(model_names)
df_t = pd.DataFrame({'inference_period': [mA] * num,
                     'domain_id': [domain_id] * num,
                     'training_samples': [len(df_domain_bh)] * num,
                     'inference_locations': [len(df_domain_infer)] * num,
                     'model': model_names,
                     'execution_time': model_exec_times,
                     'learn_time': learn_times,
                     'inference_time': inference_times})
timing_csv = f"{result_path}/timing_{domain_id}.csv"
if not os.path.exists(timing_csv):
    df_t.to_csv(timing_csv)

### Assemble predictions for all candidate models

In [ ]:
candidates_mu = OrderedDict(
                [('SK', sk_mean['raw']), ('SK_nst', sk_mean['nst']),
                 ('OK', ok_mean['raw']), ('OK_nst', ok_mean['nst']),
                 ('GP(L)', df_gpl[gpl_col_mean['raw']].values),
                 ('GP(L)_nst', df_gpl[gpl_col_mean['nst']].values),
                 ('GP(G)', df_gpg[gpg_col_mean['raw']].values),
                 ('GP(G)_nst', df_gpg[gpg_col_mean['nst']].values)
                ])
candidates_sigma = OrderedDict(
                   [('SK', np.sqrt(sk_var['raw'])), ('SK_nst', np.sqrt(sk_var['nst'])),
                    ('OK', np.sqrt(ok_var['raw'])), ('OK_nst', np.sqrt(ok_var['nst'])),
                    ('GP(L)', df_gpl[gpl_col_stdev['raw']].values),
                    ('GP(L)_nst', df_gpl[gpl_col_stdev['nst']].values),
                    ('GP(G)', df_gpg[gpg_col_stdev['raw']].values),
                    ('GP(G)_nst', df_gpg[gpg_col_stdev['nst']].values)
                   ])
for i in two_powers:
    key = f"from_{i}"
    candidates_mu[f"SK_SGS_{key}"] = sksgs_mean[key]
    candidates_sigma[f"SK_SGS_{key}"] = sksgs_stdev[key]
for i in two_powers:
    key = f"from_{i}"
    candidates_mu[f"OK_SGS_{key}"] = oksgs_mean[key]
    candidates_sigma[f"OK_SGS_{key}"] = oksgs_stdev[key]
for i in two_powers:
    key = f"from_{i}"
    candidates_mu[f"GP_SGS_{key}"] = gpsgs_mean[key]
    candidates_sigma[f"GP_SGS_{key}"] = gpsgs_stdev[key]
for i in two_powers:
    key = f"from_{i}"
    candidates_mu[f"GP_CRF_{key}"] = gpcrf_mean[key]
    candidates_sigma[f"GP_CRF_{key}"] = gpcrf_stdev[key]

# Write results to pickle file
for k in ['timing:learn', 'timing:inference']:
    _ = cfg_krige.pop(k, None)
    _ = cfg_gp.pop(k, None)

models_pfile = f"{result_path}/models-{domain_id}.p"
with open(models_pfile, 'wb') as hdl:
    variables = [candidates_mu, candidates_sigma, cfg_krige, cfg_gp]
    pickle.dump(variables, hdl, protocol=4)

## <font color="#cc0066">Part 2: Models Evaluation</font>
A _model_ refers to a probabilistic grade prediction technique such as Ordinary Kriging, Gaussian Process Regresion, and any sequential or random field simulation approaches such as OK-SGS, GP-SGS and GP-CRF. 

### Objective
For each of the model considered, we would like to
- 1) Compute <b>histogram distances</b> between the model <font color="#4F81FF"><b>mean prediction</b></font> and ground truth grade values
- 2) Compute <b>variogram ratios</b> and spatial variability of the model mean prediction relative to the ground truth
- 3) Compute the <b>goodness of model</b> <font color="#4F81FF"><b>uncertainty estimates</b></font>

These statistical measures are described below.

### <font color="#EE0066">2.1. Measuring discrepancies between model prediction and ground truth histograms</font>
Let $p$ and $q$ denote the probability mass function of the model predicted mean grade and ground truth, respectively. Let us consider four measures motivated by hypothesis testing, information theory, set theory and the Monge-Kantorovich optimal transportation (distribution morphing) problem.

#### Probabilistic symmetric Chi-square measure (Deza, 2009)
This represents twice the <i>triangular discrimination</i> defined in (Topsoe, 2000).
- $D_{psChi} = 2 \sum_x \dfrac{\left|p(x)-q(x)\right|^2}{p(x)+q(x)}$

#### Jensen-Shannon (JS) divergence
The Jensen-Shannon divergence measure, $D_{JS}$, represents the symmetric form of K divergence. It is defined by
- $\begin{align}\\
   D_{JS} &= \dfrac{1}{2} \left[ \sum_x p(x) \log\left(\dfrac{2 p(x)}{p(x)+q(x)}\right) + \sum_x q(x) \log\left(\dfrac{2 q(x)}{p(x)+q(x)}\right)\right]\\
          &= \dfrac{1}{2} [KL(p|m) + KL(q|m)]\end{align}$

In the second line, $m=(p+q)/2$. It expresses $D_{JS}$ in terms of the Kullback-Leibler distance $KL(p\mid q) = \sum_x p(x) log(p(x)/q(x))$. Using base-2 logarithm, this measure satisfies the bounds $0 \le D_{JS} \le 1$, attaining zero when $p = q$.

#### Ruzicka distance
The Ruzicka distance $D_{Ruz}$ is defined by Ruzicka similarity $S_{Ruz}$. Given two probability mass functions, $p$ and $q$,
- $D_{Ruz} = 1 - S_{Ruz} = 1 - \dfrac{\sum_x \min\{p(x),q(x)\}}{\sum_x \max\{p(x),q(x)\}}$

This may be interpreted as the `intersection(p,q)` over `union(p,q)` as a generalisation of the Jaccard similarity index from $\{0,1\}^n$ to $\mathbb{R}^n$. Ruzicka distance is bounded by $0 \le D_{Ruz} \le 1$.

#### Wasserstein distance
The Wasserstein distance $W_1$, also called the <b>Earth mover’s distance</b> or the Kantorovich <b>optimal transport distance</b>, is a similarity metric that may be interpreted as the minimum energy cost of moving and transforming a pile of dirt in the shape of one probability distribution into the other. The cost is quantified by the distance and amount of probability mass being moved. It might be preferred over JS divergence, as the Kantorovich-Mallows-Monge-Wasserstein metric represents the Lipschitz distance between probability measures and has to be K-Lipschitz continuous. When the measures are uniform over a set of discrete elements, the problem is also known as minimum weight bipartite matching.

Formally, the $p$-Wasserstein distance between probability distributions $P$ and $Q$ can be <a href="https://en.m.wikipedia.org/wiki/Earth_mover's_distance">defined</a> as an infinum over joint probabilities
- $D_{EM}(P,Q) \equiv W_p(P,Q) = \inf_{\gamma\in\Pi(P,Q)} \mathbb{E}_{(x,y)\sim\gamma}[d(x,y)]$

where $\Pi(P,Q)$ is the set of all joint distributions whose marginals are $P$ and $Q$. In general, it requires solving a <a href="https://en.wikipedia.org/wiki/Linear_assignment_problem">linear assignment problem</a>. However, in one-dimension, it can be <a href="https://en.wikipedia.org/wiki/Wasserstein_metric">obtained</a> simply using <b>order statistics</b>. In particular, $W_p(P,Q)=\left(\frac{1}{n}\sum_i^{n}\lVert X_{(i)}-Y_{(i)}\rVert^p\right)^{1/p}$. Observe that this does not require instances (predicted or true values) to be quantised or converted into histograms.

### <font color="#EE0066">2.2. Measuring loss of spatial fidelity using semi-variograms</font>
Variograms are useful for understanding spatial variability, viz., the correlation between points in space as a function of their separating distance. When the amplitude of the sill (height of the semi-variogram at large distances as samples become uncorrelated) decreases relative to a benchmark (typically, the variogram based on validation measurements at the inference locations in the block model, or training blastholes), smoothing is said to have occurred. Thus, the ratio between two variogram curves, with one being chosen as the benchmark, indicates power attenuation or a loss in spatial fidelity. Accordingly, L-statistics (such as the median and lower/upper quartiles) can be extracted from these variogram curve ratios, converting a visual diagnostic tool into a quantitative measure as shown below.
- Variogram ratios as a function of lag distance $x$:
  - $r_\text{method}(x) = \dfrac{\gamma_\text{method}(x)}{\gamma_\text{reference}(x)}$
- Median variogram ratios and lower/upper quartiles:
  - $\text{median}_x(r_\text{method}(x))$, percentile($r_\text{method}(x), [25,75])$

### <font color="#EE0066">2.3. Measuring goodness of model uncertainty estimates</font>

Measures to be described
- <b>Signed scoring function</b>, $S\equiv s(\hat{\mu},\hat{\sigma}\mid\mu_0)$,
- <b>Local Consensus</b> (a reasonableness measure) $L\equiv l(\hat{\mu},\hat{\sigma}\mid\mu_0)=\left|s \right|$
- <b>Fraction of true values contained in $p$-probability intervals</b> bounded by quantiles $\left[Q_{(1-p)/2},Q_{(1+p)/2}\right]$, $\bar{\kappa}(p)$
- <b>Accuracy</b> of the conditional distribution function, $A$, averaged over $p$
- <b>Precision</b> of the conditional distribution function, $P$, averaged over $p$
- <b>Goodness</b> statistic (Deutsch, 1997), $G$
- <b>Tightness</b> statistic, $T$, based on a slight modification of (Goovaerts, 2001).

Each model will provide a mean estimate $\hat{\mu}_j$ and standard deviation estimate $\hat{\sigma}_j$ at inference location $X_j$ where a validation measurement (true grade) $\mu_{0,j}\equiv Y_j$ is provided. Here, $(\hat{\mu}_j,\hat{\sigma}_j)$ are deemed to be sufficient for characterising the random process or conditional distribution function (cdf) which will be assumed as Gaussian distributed. For brevity, the location subscript $j$ may be dropped in subsequent discussion.

It is common to convert $(\mu_{0,j},\hat{\mu}_j,\hat{\sigma}_j)$ into a Z score,
viz $z_j = (\mu_{0,j}-\hat{\mu}_j)\,/\,\hat{\sigma}_j$. In the following figure, the black dot represents the supposed true value ($\mu_0$). The first row considers the case where the model mean _underestimates_ the true value ($\mu_0 > \hat{\mu}$). The second row represents the case where the model mean _overestimates_ the true value ($\mu_0 < \hat{\mu}$).

The shaded area in the left column represents the <font color="#cc0066">coverage probability</font>, denoted $p$. To obtain the <font color="#cc0066">local consensus</font> between model prediction and the groundtruth, we will generally be interested in its complement $1-p$ which corresponds to the tail section (shaded portion under the curve) in the right column. To distinguish overestimation from underestimation, it is convenient to introduce a signed scoring function called <font color="#cc0066">synchronicity</font>
- $s(\hat{\mu},\hat{\sigma}\mid \mu_0) \equiv s = \begin{cases}2\times\left[1 - \Phi(z)\right] & \text{if }\mu_0 \ge \hat{\mu}\\- 2 \Phi(z) & \text{otherwise}\\ \end{cases}$
- where $z=(\mu_0 - \hat{\mu})/\hat{\sigma}$ and $\Phi(z) = \frac{1}{\sqrt{2\pi}}\int_{-\infty}^{z} e^{-t^2/2} dt$ denotes the CDF of the standard normal distribution.

This may be implemented neatly using an indicator function `under_estimated = mu_hat < mu_0` so that
- `s = 2 * (under_estimated * (1 - norm.cdf(z_scores)) - (1 - under_estimated) * norm.cdf(z_scores))`
<div class="alert alert-box alert-warning">
Using the identities $\Phi(z)=\frac{1}{2}(1+\text{erf}(\frac{z}{\sqrt{2}}))$ and $\Phi_c(z)=1-\Phi(z)=\frac{1}{2}\text{erfc}(\frac{z}{\sqrt{2}})$,<br>where $\text{erf}(z)=\frac{2}{\sqrt{\pi}}\int_{0}^{z} e^{-t^2}dt$ and $\text{erfc}(z)=\frac{2}{\sqrt{\pi}}\int_{z}^{\infty} e^{-t^2}dt$,<br>we can also write 
    $s(\hat{\mu},\hat{\sigma}\mid \mu_0) \equiv s = \begin{cases}\text{erfc}(\frac{z}{\sqrt{2}}) & \text{if }\mu_0 \ge \hat{\mu}\\- (1+\text{erf}(\frac{z}{\sqrt{2}})) & \text{otherwise}\\ \end{cases}$
</div>

The local consensus term is simply given by $l(\hat{\mu},\hat{\sigma}\mid \mu_0) = \left|s\right|$
- When $\hat{\mu} = \mu_0$, the local consensus $l(\hat{\mu},\hat{\sigma}\mid \mu_0) = 1$
- In the limit as $\hat{\mu} - \mu_0 \rightarrow \pm\infty$, the local consensus $l(\hat{\mu},\hat{\sigma}\mid \mu_0) \rightarrow 0$

In [ ]:
Image(filename=os.path.join(default_data_dir, "gaussian-uncertainty-plot.png"))

### Connections with statistical measures described in the literature

#### Quality of the predicted mean 
Accuracy of the predicted mean can be measured using the mean squared error metric $MSE = \frac{1}{m}\sum_{j=1}^{m}\left|\hat{\mu} - \mu_0\right|^2$. However, this does not take into account any spatial correlation. In this work, we use <b>variogram ratios</b> to quantify the spatial variability observed in a model relative to a ground truth (based on validation measurements or training samples).
- Attention on the predicted mean and variogram ratios represents a crucial part of our model fidelity assessment. Nevertheless, we will be skipping over this to focus on the uncertainty aspect in our present discussion. Interested readers can refer to the description of variogram ratios near cell 19 in Notebook 2E to get the gist of it.

#### Goodness of the model predicted uncertainty
One criterion for assessing <b>prediction uncertainty accuracy</b> (Fouedjio and Klump, 2019) is based on the <font color="#cc0066">goodness statistic</font> (Deutsch, 1997). By construction, there is a probability $p$ (the same as the coverage probability $p$ in our earlier formulation) that the true value of the random variable falls inside the <b>symmetric p-probability interval</b> (PI) bounded by the $p_L=(1-p)/2$ and $p_U=(1+p)/2$ <b>quantiles</b> ($Q_L\equiv Q_{(1-p)/2}$ and $Q_U\equiv Q_{(1+p)/2}$) of the estimated conditional distribution function.
- As a special case, when $p=0.5$, $Q_L\equiv Q_{0.25}$ and $Q_U\equiv Q_{0.75}$ correspond to the lower and quartiles, respectively.

Given validation measurements $\{Y_j\}_{j=1,..,m}$ (or true grades $\{\mu_{0,j}\}_j$) at inference locations $\{X^{*}_j\}_j$, we are interested in the <b>fraction of true values that are bounded by the PI interval with various probability</b> $p$. Concretely,
- $\bar{\kappa}(p) = \frac{1}{m}\sum_{j=1}^m \kappa_j(p)$ defines the expectation over the locations, with
- $\kappa_j(p) =\begin{cases}1 & \text{if }\hat{Q}_{(1-p)/2}(j) < Y_j < \hat{Q}_{(1+p)/2}(j)\\0 & \text{otherwise}\\ \end{cases}$

In practice, when the random process (fluctuations about the posterior mean function) is modelled as Gaussian, we have symmetric intervals following Z-score transformation, so effectively $\hat{Q}_{(1-p)/2}(j)=-\hat{Q}_{(1+p)/2}(j)$. The mean proportion $K$ is computed by integrating $\bar{\kappa}(p)$
- $K = \int_{0}^{1} \bar{\kappa}(p) dp$

#### Accuracy of the estimated distribution
The average of $[\bar{\kappa}(p)\ge p]$ over $p$ is known as distribution accuracy.
- $A = \int_{0}^{1} \mathbb{I}(\bar{\kappa}(p)) dp$
- The indicator function $\mathbb{I}(\bar{\kappa}(p))\equiv\mathbb{I}(p)=\begin{cases}1 & \text{if }\bar{\kappa}(p) \ge p\\0 & \text{otherwise}\end{cases}$

#### Precision of the estimated distribution
Precision measures the narrowness of the model estimated distribution. It is only defined for accurate probability distributions. A $p$-probability interval that _recalls_ more than $p\,$\% of true values is accurate but not precise. Optimal precision means the $p$-PI contains the true values exactly $p\,$\% of time. On this basis, the precision of the estimated distribution when there is accuracy is defined in (<a href="#deutsch1997">Deutsch, 1997</a>) by
- $P = 1 - 2\int_{0}^{1} \mathbb{I}(p)\left[\bar{\kappa}(p)-p\right] dp$
<div class="alert alert-box alert-info">This approach relates to y-values in a <a href="#prediction-uncertainty-accuracy-plot">prediction uncertainty accuracy plot</a>. Once again, the precision of the estimated distribution is only meaningful when we have an accurate model, one where the estimated proportions $\bar{\kappa}(p)$ are consistently above the expected proportions $p$, or one with a high accuracy score ($A$).</div>
<div class="alert alert-box">A model with lower accuracy might be more selective (cautious) about the range of values that it includes, so its prediction intervals tend to be shorter. This may lead to higher precision when only samples that meet the $\mathbb{I}(p)$ criterion are considered, if $\bar{\kappa}(p)$ rises above $y=p$ at all.</div>

#### Prediction uncertainty accuracy plots<a name="prediction-uncertainty-accuracy-plot"></a>
This refers to a scatter plot of the estimated proportion $\bar{\kappa}(p)$ vs the expected proportion $p$. The model estimated CDF is considered accurate when $\bar{\kappa}(p) > p$ for all $p\in [0,1]$. <font color="#cccccc">Examples to follow later in this notebook</font>.

#### Prediction uncertainty goodness statistic
The closeness between the estimated and theoretical proportions is quantified by
- $G = 1 - \int_{0}^{1}\left[3 \mathbb{I}(p)-2\right]\left[\bar{\kappa}(p)-p\right] dp$ ......(Deutsch, 1997)

The $G$-statistic corresponds to the closeness of points to the bisector of the accuracy plot. Unlike the accuracy and precision, this also considers instances where $\bar{\kappa}(p) < p$.
- $G=1$ when $\bar{\kappa}(p) = p$, $\forall p\in [0,1]$
- $G=0$ when none of the true values are contained in any PIs
- The choice of weights (-2 vs +1) indicates that $\bar{\kappa}(p) < p$ is more consequential. The penalty for <i>below the expected</i> proportions ($\bar{\kappa}(p) < p$) is twice that for <i>above the expected</i> proportions ($\bar{\kappa}(p) > p$).

#### Width of prediction uncertainty
For models with similar goodness statistics, one would prefer a model where the p-probability interval is as narrow as possible. The tighter the intervals, the smaller the uncertainties. A model (or conditional cumulative distribution function) that consistently provides narrow and accurate PIs should be preferred over another that provides wide and accurate PIs. Different notions of spread such as entropy, variance or inter-quartile range can be used. <a href="#goovaerts2002">Goovaerts (2001)</a> proposed using the interval width to measure the average <font color="#cc0066">tightness of the p-probability intervals subject to containment of the true value</font>
- $\bar{W}(p) = \dfrac{1}{m \bar{\kappa}(p)}\sum_{j=1}^{m} \kappa_j(p)\left[\hat{Q}_{(1+p)/2}(j)-\hat{Q}_{(1-p)/2}(j)\right]$

#### Prediction uncertainty tightness statistic
At RTCMA, we define this as an average of $\bar{W}(p)$ over $p$ in a manner similar to the accuracy statistic. To make the tightness scale more meaningful, the average uncertainty interval is normalised by the process standard deviation $\sigma_Y$ observed in the validation data.
- $T = \dfrac{1}{\sigma_Y} \int_{0}^{1} \bar{W}(p) dp$

In general, both $G$ and $T$ need to be taken in account when assessing probabilistic models. "Uncertainty cannot be artificially reduced at the expense of accuracy" (Deutsch, 1997).

### Computing the histogram, variogram and uncertainty-based performance measures

In [ ]:
# General setup
mA = inference_prefix
mA_mB_mC = '%02d_%02d_%02d' % (mA, mA+1, mA+2)
default_code_dir = os.getcwd()
default_data_dir = default_code_dir.replace('code', 'data')
default_result_dir = default_code_dir.replace('code', 'results')
info_data_dir = specs.get('info:data_dir', default_data_dir)
info_result_dir = specs.get('info:result_dir', default_result_dir)
data_path = f"{info_data_dir}/{mA_mB_mC}"
subdir = check_learning_rotation_status(specs)
result_path = f"{info_result_dir}/{subdir}/{mA_mB_mC}"
figure_path = f"{info_result_dir}/{subdir}/{mA_mB_mC}/figs"
os.makedirs(figure_path, exist_ok=True)
specs.update({'mA': mA, 'domain_id': domain_id})

models_pfile = f"{result_path}/models-{domain_id}.p"
histogram_stats_pfile = f"{result_path}/histogram-stats-{domain_id}.p"
variogram_stats_pfile = f"{result_path}/variogram-stats-{domain_id}.p"
uncertainty_stats_pfile = f"{result_path}/uncertainty-stats-{domain_id}.p"
analysis_csv = f"{result_path}/analysis-{domain_id}.csv"
hdist_crossplots_file = f"{figure_path}/histogram-dist-crossplots-{domain_id}.pdf"
histogram_file = f"{figure_path}/histograms-{domain_id}.pdf"
variogram_file = f"{figure_path}/variograms-@-{domain_id}.pdf"
kappa_w_file = f"{figure_path}/uncertainty_kappa-{domain_id}.pdf"
spatial_mean_file = f"{figure_path}/spatial_mean-@-{domain_id}.pdf"
spatial_stdev_file = f"{figure_path}/spatial_stdev-@-{domain_id}.pdf"
signed_distortion_file = f"{figure_path}/uncertainty_signed_distortion-{domain_id}.pdf"
local_consensus_file = f"{figure_path}/local_consensus-{domain_id}.pdf"

if not os.path.exists(models_pfile):
    raise RuntimeError(f"Required file {models_pfile} is missing. You may need to "
                        "run `construct_models` to generate prediction results")
with open(models_pfile, 'rb') as f:
    (candidates_mu, candidates_sigma, cfg_krige, cfg_gp) = pickle.load(f)

df_bh = pd.read_csv(f"{data_path}/blastholes_tagged.csv")
df_bh = df_bh.rename(columns = {'EAST': 'X', 'NORTH': 'Y', 'RL': 'Z', 'PL_CU': 'V'})
df_domain_bh = df_bh[(df_bh['lookup_domain'] == domain_id) & np.isfinite(df_bh['V'].values)]
df_k = pd.read_csv(f"{data_path}/blocks_to_estimate_tagged.csv")
df_domain_infer = pd.DataFrame(df_k[df_k['domain'] == domain_id])
mu_0 = ground_truth = df_domain_infer['cu_bh_nn'].values
max_simul = cfg_krige['simulation:num']

### Category 1: Histogram measures for mean prediction (global accuracy)

In [ ]:
if os.path.exists(histogram_stats_pfile):
    with open(histogram_stats_pfile, 'rb') as f:
        (candidates_psChi2, candidates_JS, candidates_IOU, candidates_EM,
         candidates_hist, bins_representation, df_h) = pickle.load(f)
    stat_names = list(df_h.columns)
else:
    candidates_psChi2 = OrderedDict()
    candidates_JS = OrderedDict()
    candidates_IOU = OrderedDict()
    candidates_EM = OrderedDict()
    candidates_hist = OrderedDict()

    for k in candidates_mu.keys():
        d = compute_histogram_statistics(mu_0, candidates_mu[k])
        candidates_psChi2[k] = d['psym-chi-square']
        candidates_JS[k] = d['jensen-shannon']
        candidates_IOU[k] = d['ruzicka']
        candidates_EM[k] = d['wasserstein']
        candidates_hist[k] = d['pmf_x']

    bins_representation = d['values']
    candidates_hist['GroundTruth'] = d['pmf_y']

    # Rank models by histogram distances
    compute_rank = lambda seq: np.array([seq[k] for k in candidates_mu.keys()]).argsort().argsort() + 1
    rank_psChi2 = compute_rank(candidates_psChi2)
    rank_JS = compute_rank(candidates_JS)
    rank_IOU = compute_rank(candidates_IOU)
    rank_EM = compute_rank(candidates_EM)
    rank_mean = np.mean(np.c_[rank_psChi2, rank_JS, rank_IOU, rank_EM], axis=1)
    rank_overall = rank_mean.argsort().argsort() + 1

    data = OrderedDict()
    stat_names = ['d(psChi2)', 'd(JS)', 'd(IOU)', 'd(EM)']
    for k in candidates_mu.keys():
        data[k] = np.round([candidates_psChi2[k], candidates_JS[k], candidates_IOU[k], candidates_EM[k]], 4)
    df_h = pd.DataFrame.from_dict(data, orient='index', columns=stat_names)
    df_h['rank(psChi2)'] = rank_psChi2
    df_h['rank(JS)'] = rank_JS
    df_h['rank(IOU)'] = rank_IOU
    df_h['rank(EM)'] = rank_EM
    df_h['rank(avg)'] = rank_mean
    df_h['rank(overall)'] = rank_overall

    with open(histogram_stats_pfile, 'wb') as hdl:
        variables = [candidates_psChi2, candidates_JS, candidates_IOU, candidates_EM,
                     candidates_hist, bins_representation, df_h]
        pickle.dump(variables, hdl, protocol=4)

df_h

#### Examine cross-plots of histogram measures to check consistency
Probabilistic Symmetric Chi-square, Jensen-Shannon and Ruzicka.IOU are highly correlated $\left(\rho \gtrsim 0.975\right)$. Overall, apart from the Wasserstein (Earth Mover's) distance which is slightly different (although not necessarily inferior) to others, they exhibit a monotonic trend.
- JS divergence may be used as a substitute for  p.s. Chi-square. A nice property is that it is upper bounded.

In [ ]:
fig = plt.figure(figsize=(10,8)) #(x,y)
array_ = lambda d: np.array([d[k] for k in d.keys() if k != 'SK'])
data = [array_(candidates_psChi2), array_(candidates_JS),
        array_(candidates_IOU), array_(candidates_EM)]
stat_full_names = [r'Prob.Symm.$\chi^2$', 'Jensen-Shannon', 'Ruzicka.IOU', 'Wasserstein.EM']
stat_abbrevs = [x.strip('d()') for x in stat_names]
pairs = [(0,[1,2,3]), (1,[2,3]), (2,[3])]
for row, columns in pairs:
    for col in columns:
        plt.subplot(3,3,row*3+col)
        plt.plot([min(data[col]),max(data[col])], [min(data[row]),max(data[row])], c="#cccccc")
        plt.scatter(data[col], data[row], s=10)
        if col == columns[0]:
            plt.xlabel(stat_abbrevs[col], fontsize=8)
            plt.ylabel(stat_abbrevs[row], fontsize=8)
        else:
            plt.xticks([])
        pearson = np.corrcoef(data[row], data[col])[0,1]
        plt.text(min(data[col]), 0.6*max(data[row]), r"$\rho$={}".format('%.4f' % pearson))
        plt.title(f"{stat_full_names[row]} vs {stat_full_names[col]}", fontsize=9)
plt.savefig(hdist_crossplots_file, bbox_inches='tight', pad_inches=0.05)

#### Draw histograms for selected models

In [ ]:
# Draw histograms for selected models
nsim = min(32, 2**int(np.floor(np.log2(max_simul))))
selected = ['SK_nst', 'OK_nst', 'GP(G)_nst', 'GP(L)_nst',
            f"SK_SGS_from_{nsim}", f"OK_SGS_from_{nsim}",
            f"GP_CRF_from_{nsim}", f"GP_SGS_from_{nsim}"]
fig = plt.figure(figsize=(12,15))
w = np.median(np.diff(bins_representation))
for i, k in enumerate(selected):
    p = candidates_hist[k]
    q = candidates_hist['GroundTruth']
    plt.subplot(4,2,i+1)
    plt.bar(bins_representation, p, edgecolor=None, width=w, label=k)
    plt.bar(bins_representation, q, facecolor=None, fill=False, edgecolor='k', width=w, label='GroundTruth')
    plt.legend(fontsize=9, loc='upper left')
    if i >= len(selected)-2:
        plt.xlabel('grade')
    plt.title(f"{k} probability mass function", fontsize=10)
plt.savefig(histogram_file, bbox_inches='tight', pad_inches=0.05)

### Category 2: Variograms to measure model fidelity (local accuracy/spatial correlation)

In [ ]:
# Variogram configuration parameters and basic definitions
specs = {'variogram:linear_vertical_scale': False}

cfg_krige['variogram:use_nugget'] = specs.get('variogram:use_nugget', False)
cfg_krige['variogram:max_lag'] = specs.get('variogram:max_lag', 250.0)
cfg_krige['variogram:num_lag'] = specs.get('variogram:num_lag', 45)
cfg_krige['variogram:required_samples'] = specs.get('variogram:required_samples', 30)
vgram = {}
variogram_model = cfg_krige.get('kriging:covariance_fn', 'matern')
fixed_nu = cfg_krige.get('kriging:matern_smoothness', None)
max_range = 2 * cfg_krige['variogram:max_lag']

# Option to constrain the `nu` smoothness parameter for Matern.
# General constraints = (lower, upper) where for instance
# upper = [max_range, max_sill, max_nu, max_nugget]
constraints = None
if fixed_nu:
    constraints = ([0., 0., fixed_nu - 0.0001, 0.],
                   [max_range, max_var, fixed_nu + 0.0001, 0.5 * max_var])
    if cfg_krige['variogram:use_nugget'] is False:
        constraints = tuple([cs[:-1] for cs in constraints])

make_variogram = lambda x, y : skg.Variogram(
                 coordinates=x,
                 values=y,
                 estimator='matheron',
                 model=variogram_model,
                 use_nugget=cfg_krige['variogram:use_nugget'],
                 maxlag=max_range,
                 n_lags=cfg_krige['variogram:num_lag'],
                 fit_bounds=constraints)

class VariogramValues:
    def __init__(self, b=None, e=None):
        self.bins = b
        self.experimental = e

In [ ]:
# Compute variograms for models in the focused group
if len(ground_truth) >= cfg_krige['variogram:required_samples']:
    print(f"computing variograms", end=': ')
    # Compute variograms for candidates
    for model in candidates_mu.keys():
        x = df_domain_infer[['X','Y','Z']].values
        y = candidates_mu[model].flatten()
        retain = np.where(np.isfinite(y))[0]
        vgram[model] = make_variogram(x[retain], y[retain])
        print(f"{model}", end=', ')

    # Compute variograms for ground truth types
    for model, x, y in zip(['GroundTruth(blocks)', 'GroundTruth(training bh)'],
                           [df_domain_infer[['X','Y','Z']].values, df_domain_bh[['X','Y','Z']].values],
                           [ground_truth, df_domain_bh['V'].values]):
        retain = np.where(np.isfinite(y))[0]
        vgram[model] = make_variogram(x[retain], y[retain])
        print(f"{model}", end=', ')

    # Compute variogram stats for models declared thus far
    # - df_v.columns = ["method", "serial", "bins", "variogram($A)", "p25($A)", "p50($A)",
    #                    "p75($A)", ratios($A), "p25($B)", "p50($B)", "p75($B)", ratios($B)]
    #   where "$A"="GroundTruth(training bh)", "$B"="GroundTruth(training bh)"
    #         "method" is synonymous with "model" (such as "OK-SGS")
    #         "serial" describes the inference period and domain as f"{mA}:{domain_id}"
    ratios, stats, df_v = compute_variogram_stats(
                          vgram,
                          serial=f"{mA}:{domain_id}",
                          references=['GroundTruth(blocks)', 'GroundTruth(training bh)'],
                          percentiles=[25,50,75])

    # Compute typical variogram from single-shot simulation
    x = df_domain_infer[['X','Y','Z']].values
    for model, pred in zip(['SK_SGS', 'OK_SGS', 'GP_SGS', 'GP_CRF'],
                           [sksgs_vals, oksgs_vals, gpsgs_vals, gpcrf_vals]):
        individual_vgram = []
        for simul in np.arange(min(pred.shape[1], 16)):
            y = pred[:, simul]
            retain = np.where(np.isfinite(y))[0]
            vario = make_variogram(x[retain], y[retain])
            individual_vgram.append(vario.experimental)
        key = model + '_single'
        vgram[key] = VariogramValues(vario.bins, np.mean(individual_vgram, axis=0))
        print(f"{key}", end=', ')

    # Compute variograms for remaining "black box" models
    for model, x, y in zip(['RTK_LongRangeModel'],
                           [df_domain_infer[['X','Y','Z']].values] * 1,
                           [df_domain_infer['cu_50'].values]):
        retain = np.where(np.isfinite(y))[0]
        vgram[model] = make_variogram(x[retain], y[retain])
        print(f"{model}", end=', ')

    with open(variogram_stats_pfile, 'wb') as hdl:
        pickle.dump([ratios, stats, df_v], hdl, protocol=4)
else:
    models = list(candidates_mu.keys()) + ['GroundTruth(blocks)', 'GroundTruth(training bh)']
    vgram_ratios50 = [np.nan] * len(models)
    df_v = pd.DataFrame(zip(models, vgram_ratios50), columns=['method','p50(GroundTruth(blocks))'])

In [ ]:
# Generate variogram plots using a consistent colour scheme
# Curves for model <m> consist of
#    ["<m>_SGS_single"] +                      #R.1
#    ["<m>_SGS_from{t}" for t in two_powers] + #R.2
#    ["<m>_nst"] if <m> ! 'GP(L)' else [] +    #R.3
#    ['GP(L)_nst] +                            #R.4
#    ["RTK_LongRangeModel"] +                  #R.5
#    ["GroundTruth(blocks)", "GroundTruth(training bh)"] #R.6
two_powers = [int(k.split('_')[-1]) for k in candidates_mu if 'CRF_from' in k]
reserved_patterns = ['-<','->','-x','-o','--*','-.p',':P',':s']
n = len(two_powers)
clut = lambda words: [variograms_colour_lookup(w) for w in words]
common = ['lilac', 'gray'] + ['black']*2 #covers R.4-R.6
palette = {'SK_SGS': clut(['olive'] + ['green-mid']*n + ['green-dark'] + common),
           'OK_SGS': clut(['magenta'] + ['blue-light']*n + ['blue-dark'] + common),
           'GP_SGS': clut(['red-dark'] + ['red']*n + common),
           'GP_CRF': clut(['orange-dark'] + ['orange']*n + ['pink'] + common)
          }

if len(ground_truth) >= cfg_krige['variogram:required_samples']:
    vgram_peak = max([np.nanmax(v.experimental) for k,v in vgram.items()])
    vgram_trough = min([np.nanmin(v.experimental) for k,v in vgram.items()])
    linear_vertical_scale = specs.get('variogram:linear_vertical_scale', False)
    if not linear_vertical_scale:
        vgram_trough = np.maximum(1e-4, vgram_trough)
    category_items = OrderedDict()

    categories = ['SK_SGS', 'OK_SGS', 'GP_SGS', 'GP_CRF']
    base_techniques = ['SK_nst', 'OK_nst', 'GP(L)_nst', 'GP(G)_nst']
    for base, cat in zip(base_techniques, categories):
        print(f"Producing variogram plot for {cat}")
        category_items[cat] = [f"{cat}_single"] \
                            + [x for x in vgram.keys() if f"{cat}_from" in x] \
                            + [base] + (['GP(L)_nst'] if base != 'GP(L)_nst' else []) \
                            + ['RTK_LongRangeModel'] \
                            + ['GroundTruth(blocks)', 'GroundTruth(training bh)']
        patterns = ['-^'] + reserved_patterns[:n] \
                 + (['-o'] if base != 'GP' else []) + ['-o','-o','-o','-o','--o']
        colors = palette[cat]
        plt.figure(figsize=(10,8))
        for i, model in enumerate(category_items[cat]):
            xe = vgram[model].bins
            ye = vgram[model].experimental
            if linear_vertical_scale:
                plt.plot(xe, ye, patterns[i], color=colors[i], markersize=4, label=model)
                legend_position = 'upper left'
            else:
                plt.semilogy(xe, ye, patterns[i], color=colors[i], markersize=4, label=model)
                legend_position = 'lower right'
        plt.title(f"Variograms comparison (models in the {cat} family)")
        plt.ylim([vgram_trough, np.ceil(1000 * vgram_peak) / 1000])
        plt.xlabel('Lag [m]')
        plt.ylabel('Semivariance')
        plt.legend(loc=legend_position)
        plt.savefig(variogram_file.replace('@', cat.lower()), bbox_inches='tight', pad_inches=0.05)

### Category 3: Uncertainty-based measures
### Evaluate performance of probabilistic models

In [ ]:
# Specify a default p vector with 256 bins
# - comprising two linear segments with denser spacing in the upper echelon
default_p_values = np.r_[np.linspace(0,0.98,247)[1:-1], np.linspace(0.98,1,12)[:-1]]
p_values = specs.get('uncertainty:p_values', default_p_values)

if os.path.exists(uncertainty_stats_pfile):
    with open(uncertainty_stats_pfile, 'rb') as f:
        (candidates_s, candidates_L, candidates_K, candidates_A,
         candidates_P, candidates_G, candidates_W, candidates_T,
         df_a, df_s) = pickle.load(f)
        df_s.rename(columns={'Likelihood': 'Consensus'}, inplace=True)    stat_names = list(df_s.columns)
else:
    # Investigate the sensitivity of kappa accuracy measure
    candidates_A_slack = OrderedDict()
    keys_a = []
    for xi in [0, 0.005, 0.01, 0.05, 0.1, 0.25]:
        keys_a.append(f"Accuracy({xi})")
        candidates_A_slack[xi] = OrderedDict()
        for k in candidates_mu.keys():
            s_scores = compute_model_consensus(mu_0, candidates_mu[k], candidates_sigma[k])[1]
            accuracy = compute_distribution_accuracy(p_values, None, s_scores, slack=xi)
            candidates_A_slack[xi][k] = accuracy

    data_a = OrderedDict()
    for k in candidates_mu.keys():
        data_a[k] = np.round([d[k] for s, d in candidates_A_slack.items()], 4)
    df_a = pd.DataFrame.from_dict(data_a, orient='index', columns=keys_a)

df_a

#### Consider the sensitivity of the kappa accuracy measure
Reflecting on the definition of distribution accuracy, the original definition does seem to be unnecessarily strict. In the `Accuracy(0)` column, the maximum accuracy attained by some models frequently lies below 0.8.
- The accuracy score, $A = \int_{0}^{1} [\bar{\kappa}(p)\ge p]\,dp$, where $\bar{\kappa}(p) = \frac{1}{m}\sum_{j=1}^m \kappa_j(p)$ depends on
- $\kappa_j(p) =\begin{cases}1 & \text{if }\hat{Q}_{(1-p)/2}(j) < Y_j < \hat{Q}_{(1+p)/2}(j) \iff p \ge p_j^{*}\text{ where }p_j^{*} = 1 - l(\hat{\mu_j},\hat{\sigma_j}\mid\mu_{0,j})\\0 & \text{otherwise}\\ \end{cases}$

Very often, $\bar{\kappa}(p) \gtrsim p$ is close to or in the vicinity of $p$. This inequality condition can be relaxed by introducing a slack variable $\xi$, such that
- $A_{\xi} = \int_{0}^{1} [\bar{\kappa}(p)\ge (1-\xi)p]\,dp$

with typical values of $1-\xi\in [0.9,1]$ to better reflect the accuracy of distribution in the (Deutsch, 1997) sense.

In [ ]:
if not os.path.exists(uncertainty_stats_pfile):
    # Compute uncertainty-based measures
    # - s, L and K denote the signed affinity, local consensus, and mean kappa proportions
    # - A, P, G, W, and T denote the accuracy, precision, goodness, width and tightness
    #   of the conditional distribution function from a given model
    candidates_s = OrderedDict()
    candidates_L = OrderedDict()
    candidates_K = OrderedDict()
    candidates_A = OrderedDict()
    candidates_P = OrderedDict()
    candidates_G = OrderedDict()
    candidates_W = OrderedDict()
    candidates_T = OrderedDict()
    for k in candidates_mu.keys():
        d = compute_performance_statistics(mu_0, candidates_mu[k], candidates_sigma[k], slack=0.05)
        candidates_s[k] = d['s_scores']
        candidates_L[k] = np.mean(d['consensus'])
        candidates_K[k] = d['proportion']
        candidates_A[k] = d['accuracy']
        candidates_P[k] = d['precision']
        candidates_G[k] = d['goodness']
        candidates_W[k] = d['width']
        candidates_T[k] = d['tightness']

    data_precise = OrderedDict()
    data_rounded = OrderedDict()
    stat_names = ['h(psChi2)', 'h(JS)', 'h(IOU)', 'h(EM)', 'h(rank)',
                  'Variogram Ratios', 'Spatial Fidelity', '|s|_L', '|s|_U', 'Consensus',
                  'Accuracy(.05)', 'Precision', 'Goodness', 'Tightness']
    for k in candidates_mu.keys():
        s = np.abs(candidates_s[k])
        variogram_ratio = df_v[df_v.method==k]['p50(GroundTruth(blocks))'].values[0]
        spatial_fidelity = np.sqrt(1 - np.abs(np.minimum(variogram_ratio, 2) - 1))
        record = [candidates_psChi2[k], candidates_JS[k], candidates_IOU[k],
                  candidates_EM[k], df_h.loc[k,'rank(overall)'],
                  variogram_ratio, spatial_fidelity,
                  np.percentile(s,25), np.percentile(s,75), candidates_L[k],
                  candidates_A[k], candidates_P[k], candidates_G[k], candidates_T[k]]
        data_precise[k] = record
        data_rounded[k] = np.round(record, 4)
    df_s = pd.DataFrame.from_dict(data_precise, orient='index', columns=stat_names)
    df_s.to_csv(analysis_csv)

    with open(uncertainty_stats_pfile, 'wb') as hdl:
        variables = [candidates_s, candidates_L, candidates_K, candidates_A,
                     candidates_P, candidates_G, candidates_W, candidates_T, df_a, df_s]
        pickle.dump(variables, hdl, protocol=4)

df_s.round(4)

### Graphs - Prediction uncertainty accuracy and width

In [ ]:
# Produce graphs on prediction uncertainty accuracy and interval width
p_vals = np.r_[np.linspace(0,1,41)[1:-1], 0.9825, 0.99, 0.997]
alpha = np.linspace(0.025, 0.99, 100)
n_sigma = norm.ppf(1 - (1 - np.array(alpha))/2, 0, 1)

fig = plt.figure(figsize=(12,30))
for i, k in enumerate(selected):
    s_scores = candidates_s[k]
    sigma_hat = candidates_sigma[k]
    signal_variance = np.var(ground_truth)
    kappa_scores = compute_kappa_bar(s_scores, p_vals)
    interval_widths = compute_width_bar(sigma_hat, s_scores, p_vals)
    # kappa (accuracy) plots
    plt.subplot(8,2,2*i+1)
    plt.plot(p_vals, p_vals, '-', color="#888888")
    plt.plot(p_vals, kappa_scores, 'k')
    plt.ylim([0,1])
    plt.text(0.375, 0.25, "Distribution accuracy: A=%.6f" % candidates_A[k], fontsize=9)
    plt.text(0.375, 0.175, "Coverage precision: P=%.6f" % candidates_P[k], fontsize=9)
    plt.text(0.375, 0.1, "Deutsch goodness statistic: G=%.6f" % candidates_G[k], fontsize=9)
    plt.legend(["expected proportions", r"$\kappa(\hat{\mu},\hat{\sigma}\mid\mu_0$)"], loc='upper left')
    plt.title(f"Kappa plot - Expected vs estimated proportions for {k}", fontsize=10)
    if i >= len(selected) - 1:
        plt.xlabel('p')
    # W plots
    plt.subplot(8,2,2*i+2)
    n_sigma = norm.ppf(1 - (1 - np.array(alpha))/2, 0, 1)
    plt.plot(alpha, n_sigma, '-', color="#888888")
    plt.plot(p_vals, interval_widths / np.sqrt(signal_variance), 'k')
    plt.ylim([0,2])
    plt.text(0.0, 1.4, "Black curve: the lower the better", fontsize=9, color="#888888")
    plt.text(0.25, 0.1, r"Tightness statistic: T=%.6f [for $\kappa_j(p)>p]$" % candidates_T[k], fontsize=9)
    plt.legend(["norminv(1-(1-p)/2)", r"Width of prediction interval: $\bar{W}(p)/\sigma_Y$"], loc='upper left')
    plt.title(f"W plot - normalised predict interval width vs p for {k}", fontsize=10)
    if i >= len(selected) - 1:
        plt.xlabel('p')    
plt.savefig(kappa_w_file, bbox_inches='tight', pad_inches=0.05)

#### Comments
We continue to use the prediction interval width
- $\bar{W}(p) = \dfrac{1}{m \bar{\kappa}(p)}\sum_{j=1}^{m} \kappa_j(p)\left[\hat{Q}_{(1+p)/2}(j)-\hat{Q}_{(1-p)/2}(j)\right]$
as defined by (Goovaerts, 2001).

However, we incorporate a signal variance normalisation in our tightness definition
- $T = \dfrac{1}{\sigma_Y} \int_{0}^{1} \bar{W}(p) dp$,
where $\sigma_Y$ denotes the standard deviation observed in the validation data.

If we write $\bar{W}'(p) = \dfrac{\bar{W}(p)}{\sigma_Y}$, this allows us to compare the $\bar{W}'$ curves as a function of $p$ with the inverse normal cdf, which illustrates more clearly the <font color="#cc0066">predictive confidence</font> of a model relative to the variability of the underlying process.

### Visualisation of model estimates $\mu$ and $\sigma$ 

In [ ]:
fig = plt.figure(figsize=(12,12))
print("Predicted mean comparison for main model candidates")
for i, k in enumerate(selected):
    cbar_desc = "mu(Cu)"
    hide_x_axis = False if i >= len(selected)-2 else True
    hide_y_axis = False if i%2==0 else True
    make_scatter_2d(
        df_domain_infer['X'], df_domain_infer['Y'], candidates_mu[k],
        min_v, max_v, symbsiz=50, subplotargs=[4,2,i+1], palette='YlOrRd', cbtitle=cbar_desc,
        sharex=hide_x_axis, sharey=hide_y_axis, symbol='s',
        graphtitle=f"{k} predicted mean")
plt.savefig(spatial_mean_file.replace('@', 'for_candidates'), bbox_inches='tight', pad_inches=0.05)

print("Predicted mean as a function of number of simulations")
for cat in ['OK_SGS', 'GP_SGS', 'GP_CRF']:
    fig = plt.figure(figsize=(12,12))
    for i, m in enumerate(two_powers):
        make_scatter_2d(
            df_domain_infer['X'], df_domain_infer['Y'], candidates_mu[f"{cat}_from_{m}"],
            min_v, max_v, symbsiz=50, subplotargs=[4,2,i+1], palette='YlOrRd', cbtitle="mu(Cu)",
            sharex=True, symbol='s', graphtitle=f"{cat} predicted Cu mean(from {m} runs)")
    plt.savefig(spatial_mean_file.replace('@', f"{cat}_convergence"), bbox_inches='tight', pad_inches=0.05)

fig = plt.figure(figsize=(12,12))
print("Predicted standard deviation comparison for main model candidates")
for i, k in enumerate([m for m in selected if m not in ['SK','OK']]): #code based on notebook 2D.7
    cbar_desc = "sigma(Cu)"
    hide_x_axis = False if i >= len(selected)-2 else True
    hide_y_axis = False if i%2==0 else True
    ymin = np.percentile(candidates_sigma[k], 5) if k in ['SK','OK'] else min_vsd
    ymax = np.percentile(candidates_sigma[k], 95) if k in ['SK','OK'] else max_vsd
    make_scatter_2d(
        df_domain_infer['X'], df_domain_infer['Y'], candidates_sigma[k],
        ymin, ymax, symbsiz=50, subplotargs=[4,2,i+1], palette='Blues', cbtitle=cbar_desc,
        sharex=hide_x_axis, sharey=hide_y_axis, symbol='s',
        graphtitle=f"{k} predicted stdev")
plt.savefig(spatial_stdev_file.replace('@', 'for_candidates'), bbox_inches='tight', pad_inches=0.05)

print("Predicted standard deviation as a function of number of simulations")
for cat in ['OK_SGS', 'GP_SGS', 'GP_CRF']:
    fig = plt.figure(figsize=(12,12))
    for i, m in enumerate(two_powers):
        make_scatter_2d(
            df_domain_infer['X'], df_domain_infer['Y'], candidates_sigma[f"{cat}_from_{m}"],
            min_vsd, max_vsd, symbsiz=50, subplotargs=[4,2,i+1], palette='Blues', cbtitle="sigma(Cu)",
            sharex=True, symbol='s', graphtitle=f"{cat} predicted Cu stdev(from {m} runs)")
    plt.savefig(spatial_stdev_file.replace('@', f"{cat}_convergence"), bbox_inches='tight', pad_inches=0.05)

### Visualise Synchronicity: $s(\hat{\mu},\hat{\sigma}\mid\mu_0)$

In [ ]:
cmapBlRd = create_inverted_colormap(gamma=2.5)
fig = plt.figure(figsize=(12,15))
print('Using s score to illustrate relative distortion')
print('Visual interpretation: Map of questionable errors or least probable predictions')
print(' - Dark: bad, light: good')
print(' - Red: under-estimated, blue: over-estimated')
displayed_models = [m for m in selected if m!='OK' and m!='SK']
for i, k in enumerate(displayed_models):
    cbar_desc = "red=under-estimated, 0=worst" if i%2==1 else "quality"
    hide_x_axis = False if i >= len(displayed_models)-2 else True
    hide_y_axis = False if i%2==0 else True
    make_scatter_2d(
        df_domain_infer['X'], df_domain_infer['Y'], candidates_s[k],
        -1, +1, symbsiz=50, subplotargs=[4,2,i+1], palette=cmapBlRd, cbtitle=cbar_desc,
        sharex=hide_x_axis, sharey=hide_y_axis, symbol='s',
        graphtitle=r"Distortion Map, $s(\hat{\mu},\hat{\sigma}\mid\mu_0)$ for " + f"{k}")
plt.savefig(signed_distortion_file, bbox_inches='tight', pad_inches=0.05)

cmapBl = create_inverted_colormap(gamma=0.7, monochrome=True)
fig = plt.figure(figsize=(12,15))
print('Using consensus score to illustrate quality of probabilistic prediction')
for i, k in enumerate(displayed_models):
    hide_x_axis = False if i >= len(displayed_models)-2 else True
    hide_y_axis = False if i%2==0 else True
    make_scatter_2d(
        df_domain_infer['X'], df_domain_infer['Y'], np.abs(candidates_s[k]),
    0, +1, symbsiz=50, subplotargs=[4,2,i+1], palette=cmapBl, cbtitle="consensus",
    sharex=hide_x_axis, sharey=hide_y_axis, symbol='s',
    graphtitle=r"Local consensus $l(\hat{\mu},\hat{\sigma}\mid\mu_0)$ for " + f"{k}")
plt.savefig(local_consensus_file, bbox_inches='tight', pad_inches=0.05)